<a href="https://colab.research.google.com/github/vladimirwest/DeOldify_universal_colab/blob/master/deoldify_universal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#◢ DeOldify - Colorize your own photos!

####**Credits:**
Original repo - https://github.com/jantic/DeOldify  
Used repo - https://github.com/tg-bomze/ENTAR  
My github [@vladimirwest](https://github.com/vladimirwest)

This is universial colab notebook for videos, photos and photos archieves conversion.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


In [0]:
#@title ##**Libraries and environment installation** { display-mode: "form" }
#@markdown *Download pre-trained models and clone the repo.*


!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify
#NOTE:  This must be the first call in order to work properly!
from deoldify import device
from deoldify.device_id import DeviceId
#choices:  CPU, GPU0...GPU7
device.set(device=DeviceId.GPU0)

import torch

if not torch.cuda.is_available():
    print('GPU not available.')

!pip install -r colab_requirements.txt

import fastai
from deoldify.visualize import *
from IPython.display import clear_output
from google.colab import files
from pathlib import Path

clear_output()
print('Done!')

In [0]:
#@title ##**File load** { display-mode: "form" }
#@markdown *Enter a link to the file below (for example, YouTube video), or leave the ** source_url ** field blank (in this case, you will be prompted to download the file from your computer).*  
#  @markdown Supported types:
#@markdown *  .zip - archive with pictures
#@markdown *  .mp4
#@markdown *  .png or .jpg
#  @markdown 
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  file_ext = fn.split(".")[-1]
  file_name = "downloaded." + file_ext
  if(file_ext =='jpg' or file_ext == 'png'):
    file_type = 'image'
  elif(file_ext =='mp4'):
    file_type = 'video'
  elif(file_ext =='zip'):
    file_type = 'zip'
  else:
    file_type = 'other'
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded.mp4'
    file_type = 'video'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    file_ext = fn.split(".")[-1]
    file_name = "downloaded." + file_ext
    !mv -f $fn $file_name
    if(file_ext =='jpg' or file_ext == 'png'):
      file_type = 'image'
    elif(file_ext =='mp4'):
      file_type = 'video'
    elif(file_ext =='zip'):
      file_type = 'zip'
    else:
      file_type = 'other'

clear_output()
print(file_type)
print('Done!')

if not os.path.exists('models'):
  if(file_type == 'image' or file_type == 'zip'):
    !mkdir 'models'
    print('Downloading model weights..')
    !wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth
  elif(file_type == 'video'):
    !mkdir 'models'
    print('Downloading model weights..')
    !wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
clear_output()
print('Done!')


In [0]:
#@title ##**Crop video (Optional block)** { display-mode: "form" }
if(file_type == 'video'):
  import moviepy.editor as mpy
  #@markdown *Here you can crop the video if it is too long.*

  #@markdown **Enter the time frame for croping the video**
  target_start = '00:00:00' #@param {type:"string"}
  target_end = '00:00:10' #@param {type:"string"}

  #@markdown Enter a new resolution of the video file for rescaling (*for example, 640 and 480*), or leave it blank to work with the original resolution
  width = '' #@param {type:"string"}
  height = '' #@param {type:"string"}

  rescale = ""
  if width != '' and height != '':
    rescale = f"-s {width}x{height}"

  !ffmpeg -i downloaded.mp4 $rescale -ss $target_start -to $target_end new_target.mp4
  clear_output()
  print('Done!')
  !mv new_target.mp4 downloaded.mp4
  #@markdown *Display video:*
  display_video = True #@param {type:"boolean"}
  if display_video == True:
    display(mpy.ipython_display("downloaded.mp4", height=360))
else:
  print('File is not video, skipping..')

In [0]:
#@title ##**Colorize:** { display-mode: "form" }
from os import listdir
#@markdown *Download converted file:*
download_file = True #@param {type:"boolean"} 
if(file_type == 'image'):
  colorizer = get_image_colorizer(artistic=True)
  clear_output()
  render_factor = 35
  image_path = colorizer.plot_transformed_image(path=file_name, render_factor=render_factor, compare=True)
  show_image_in_notebook(image_path)
  if(download_file):
    !zip -r -o converted_image.zip $image_path
  files.download('converted_image.zip')

elif(file_type == 'video'):
  !rm -rf video
  !mkdir 'video'
  !mkdir 'video/source'
  !mv downloaded.mp4 video/source/downloaded.mp4
  colorizer = get_video_colorizer()
  clear_output()
  render_factor = 21
  video_path = colorizer.colorize_from_file_name(file_name, render_factor=render_factor, watermarked = False, post_process = False)
  show_video_in_notebook(video_path)
  if(download_file):
    !mv video/result/downloaded.mp4 video/result/converted_video.mp4
    !zip -r -o converted_video.zip video/result/converted_video.mp4
    files.download('converted_video.zip')

elif(file_type == 'zip'):
  colorizer = get_image_colorizer(artistic=True)
  render_factor = 35
  clear_output()
  print('Unzipping files, please wait')
  !unzip -o $file_name -d 'images_folder'
  types = ('*.jpg', '*.png') # the tuple of file types
  files_grabbed = []
  for files_ in types:
    files_grabbed.extend(Path('images_folder').rglob(files_))
  for file_ in files_grabbed:
    image_path = colorizer.plot_transformed_image(path=file_, render_factor=render_factor, compare=True)
  #show_image_in_notebook(image_path)
  !zip -r -o converted_images.zip result_images
  files.download('converted_images.zip')

else:
  print('Wrong file extension, please try another file!')  


